# 03 — Pré-processamento · Film Greenlight Recommender

**Objetivo** — Padronizar tipos, tratar ausências e derivar variáveis **tabulares**
numéricas e categóricas em um **pipeline reprodutível** (sem usar texto por enquanto).

**Entradas** — `data/raw/imdb.csv`  
**Saídas** — `models/preprocessor_tabular.pkl`, `models/meta/*.json`,  
`data/processed/tabular_prep.npy` (*ou* `tabular_prep_csr.npz` se esparso)

**Escopo (tabular)**  
- Conversões: `Runtime_min`, `Gross_usd`, `Released_Year → Decade`.  
- Derivações: `log1p(No_of_Votes)`, `log1p(Gross_usd)`.  
- Tratamento de nulos: `Meta_score`, `Certificate`, `Gross_usd`, etc.  
- Codificação: One-Hot (`Certificate`, `Decade`).  
- Escalonamento: `StandardScaler` em numéricas.  
- Gêneros: multi-hot (filtro por suporte mínimo).

**Decisões principais**  
- *Leakage*: todas as transformações ficam **dentro** do pipeline.  
- *Reprodutibilidade*: `SEED` fixo.  
- *Robustez*: logs para caudas longas; imputadores explícitos.

Autora: *Ana Luiza Gomes Vieira* · Execução: *Set/2025*

# Setup

In [ ]:
# Bibliotecas padrão
from pathlib import Path
import re

# Bibliotecas de terceiros
import joblib
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV, KFold, cross_validate, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [2]:
# Configurações globais
SEED = 42
PROJECT_ROOT = Path.cwd().parent
DATA_RAW     = PROJECT_ROOT / "data" / "raw" / "imdb.csv"
MODELS_DIR   = PROJECT_ROOT / "models"
REPORTS_DIR  = PROJECT_ROOT / "reports"
FEATURES_DIR = PROJECT_ROOT / "data" / "processed"
for p in (MODELS_DIR, REPORTS_DIR, FEATURES_DIR):
    p.mkdir(parents=True, exist_ok=True)

assert DATA_RAW.exists(), f"Arquivo não encontrado: {DATA_RAW}"

In [3]:
df_raw = pd.read_csv(DATA_RAW)
df = df_raw.copy()

## 1. Helpers

In [4]:
def to_minutes(runtime: str) -> float:
    m = re.search(r"(\d+)", str(runtime))
    return float(m.group(1)) if m else np.nan

def to_usd(gross: str) -> float:
    s = str(gross).replace(",", "").strip()
    return float(s) if s and s.lower() != "nan" else np.nan

def to_decade(year) -> float:
    y = pd.to_numeric(year, errors="coerce")
    return np.floor(y/10)*10 if pd.notnull(y) else np.nan

## 2. Conversões & Derivações

In [5]:
df["Runtime_min"]     = df["Runtime"].apply(to_minutes)
df["Gross_usd"]       = df["Gross"].apply(to_usd)
df["Released_Year"]   = pd.to_numeric(df["Released_Year"], errors="coerce")
df["Decade"]          = df["Released_Year"].apply(to_decade)
df["No_of_Votes_log"] = np.log1p(df["No_of_Votes"])
df["Gross_usd_log"]   = np.log1p(df["Gross_usd"])

## 3. Gêneros em Multi-hot (com filtro de suporte)
Foi mantido apenas gêneros com **suporte mínimo** para evitar alta esparsidade em classes raras.

In [6]:
genre_dum = df["Genre"].str.get_dummies(sep=", ").astype(int)
min_support = 20
keep_genres = genre_dum.columns[genre_dum.sum() >= min_support].tolist()
genre_dum = genre_dum[keep_genres] 

## 4. Definição de Colunas (numéricas e categóricas)

In [7]:
num_cols_base = [
    "Runtime_min",
    "Meta_score",
    "No_of_Votes", "No_of_Votes_log",
    "Gross_usd",   "Gross_usd_log",
]
cat_cols = ["Certificate", "Decade"]

In [8]:
num_cols = num_cols_base + keep_genres

In [9]:
features_df = pd.concat([df[num_cols_base], genre_dum, df[cat_cols]], axis=1)

## 5. ColumnTransformer (imputação, escala e OHE)

In [ ]:
num_pipeline = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])
cat_pipeline = Pipeline([
    ("imp", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=True)),
])

preprocessor_tab = ColumnTransformer(
    transformers=[
        ("num", num_pipeline, num_cols),
        ("cat", cat_pipeline, cat_cols),
    ],
    remainder="drop",
    verbose_feature_names_out=False,
)

## 6. Fit/Transform & Sanity Checks

In [11]:
X_prep = preprocessor_tab.fit_transform(features_df)
print("[ok] shape X_prep:", X_prep.shape)

[ok] shape X_prep: (999, 51)


In [12]:
expected = set(num_cols + cat_cols)
present  = set(features_df.columns)
missing = expected - present
if missing:
    raise ValueError(f"Colunas ausentes: {sorted(missing)}")

## 7. Persistência de Artefatos (preprocessador, metadados e matriz)

In [13]:
joblib.dump(preprocessor_tab, MODELS_DIR / "preprocessor_tabular.pkl")

import json
(Path(MODELS_DIR) / "meta").mkdir(parents=True, exist_ok=True)
with open(MODELS_DIR / "meta" / "keep_genres.json", "w", encoding="utf-8") as f:
    json.dump(keep_genres, f, ensure_ascii=False, indent=2)

with open(MODELS_DIR / "meta" / "num_cols_base.json", "w", encoding="utf-8") as f:
    json.dump(num_cols_base, f, ensure_ascii=False, indent=2)

with open(MODELS_DIR / "meta" / "cat_cols.json", "w", encoding="utf-8") as f:
    json.dump(cat_cols, f, ensure_ascii=False, indent=2)

print("[ok] preprocessor + metadados salvos.")

[ok] preprocessor + metadados salvos.


In [15]:
from scipy import sparse
import numpy as np

if sparse.issparse(X_prep):
    sparse.save_npz(FEATURES_DIR / "tabular_prep_csr.npz", X_prep)
    print("[ok] matriz esparsa salva em data/processed/tabular_prep_csr.npz")
else:
    np.save(FEATURES_DIR / "tabular_prep.npy", X_prep)
    print("[ok] matriz densa salva em data/processed/tabular_prep.npy")

[ok] matriz densa salva em data/processed/tabular_prep.npy


# Justificativa do Pré-processamento (Tabular)

> Documentação das escolhas de pré-processamento do notebook **03 — Pré-processamento**, com base nos achados do EDA e do Overview. Texto em registro impessoal.

## Objetivos e princípios

- **Modularidade e reprodutibilidade**: centralizar conversões, imputações e codificações em um único `ColumnTransformer` (persistido em `models/preprocessor_tabular.pkl`).
- **Prevenção de vazamento (leakage)**: transformações ajustadas apenas nos dados de treino quando usadas em modelagem via `Pipeline`.
- **Robustez a nulos e caudas longas**: estratégias compatíveis com a distribuição observada e com a presença de faltantes.

---

## Conversões e derivações

- **`Runtime → Runtime_min`**  
  Extração dos minutos de strings do tipo `"142 min"`, viabilizando uso como variável numérica.

- **`Gross → Gross_usd`**  
  Remoção de vírgulas e conversão para `float`. A coluna apresenta ~17% de faltantes; imputação realizada no pipeline (mediana) para evitar descarte de linhas.

- **`Released_Year → Decade`**  
  Substituição do ano bruto pela **década**. A decisão reduz o risco de induzir relação espúria “quanto mais recente, melhor” e captura tendências temporais coerentes com o EDA.

- **`No_of_Votes_log = log1p(No_of_Votes)` e `Gross_usd_log = log1p(Gross_usd)`**  
  Transformações logarítmicas aplicadas devido às **caudas longas** identificadas. Reduzem assimetria, aproximam relações lineares e estabilizam variância.  
  Mantêm-se **ambas as versões** (bruta e log) para permitir que os modelos selecionem a escala mais informativa.

---

## Categóricas e multi-label

- **`Certificate` e `Decade`**  
  Variáveis de **baixa cardinalidade** (conforme EDA). Codificação por **One-Hot** com `handle_unknown="ignore"` para tolerância a categorias inéditas no treino.

- **`Genre` (multi-label) → multi-hot**  
  Expansão por `str.get_dummies(sep=", ")`, uma vez que um filme pode pertencer a vários gêneros.  
  **Filtro de suporte**: retenção apenas de gêneros com **≥ 20** ocorrências para reduzir esparsidade e ruído de classes raras (compatível com a dispersão observada).

---

## Tratamento de valores ausentes

- **Numéricas**: `SimpleImputer(strategy="median")` — mediana é estável sob outliers e compatível com distribuições assimétricas.  
- **Categóricas**: `SimpleImputer(strategy="most_frequent")` — opção simples e eficaz para variáveis discretas como `Certificate` e `Decade`.  
- **Cascata com logs**: faltantes em `Gross_usd` implicam `NaN` em `Gross_usd_log`; ambas são imputadas no pipeline, evitando perda de observações.

---

## Escalonamento

- **`StandardScaler`** aplicado às variáveis numéricas (incluindo dummies 0/1).  
  A padronização favorece modelos com regularização (p. ex., ElasticNet), equalizando magnitudes sem alterar a informação binária.

---

## Consistência entre treino e inferência

- Persistência de metadados em `models/meta/` (`keep_genres.json`, `num_cols_base.json`, `cat_cols.json`).  
  Garante reconstrução da **mesma matriz de features** em produção (ordem de colunas e conjunto de gêneros), evitando erros de alinhamento.

---

## Escopo deliberadamente tabular

- O texto (*Overview*) permanece fora deste pipeline. A opção visa **modularidade**: NLP (TF-IDF, modelos e tuning) encontra-se documentado no notebook específico e pode ser acoplado posteriormente via `ColumnTransformer`.

---

## Alinhamento com o EDA/Overview

- **Caudas longas** em `Gross` e `No_of_Votes` → uso de `log1p`.  
- **Tendências temporais** melhor capturadas por **década** do que por ano.  
- **Gênero multi-label** com **muitas classes raras** → multi-hot com **limiar de suporte**.  
- **Faltantes relevantes** (p. ex., `Gross`, `Meta_score`, `Certificate`) → imputação explícita e transparente.

---

## Riscos e mitigação

- **Classes raras** nos gêneros podem introduzir ruído → mitigado por **limiar de suporte** e regularização na modelagem.  
- **Imputação** pode enviesar se os nulos não forem aleatórios → possível extensão com **indicadores de imputação** (flags 0/1) em versões futuras.  
- **Escalonamento de dummies** pode causar estranheza conceitual → não altera a informação; padroniza magnitudes para regularização.

---

## Métricas e avaliação (ligação com modelagem)

- Para previsão de **`IMDB_Rating`** (problema de **regressão**), adotam-se **RMSE** (sensível a grandes erros) e **MAE** (robusto a outliers) como métricas complementares. A escolha reflete a natureza contínua da variável-alvo, com faixa relativamente estreita observada no EDA.

---

## Benefícios do desenho adotado

- **Pipeline único** e versionável (`joblib`), com **reprodutibilidade** assegurada (SEED e metadados).  
- **Menor viés de escala** e **menor sensibilidade a outliers** via log e padronização.  
- **Representação adequada de gêneros** (multi-label) sem explosão desnecessária de dimensionalidade, graças ao filtro de suporte.

